In [8]:
import pandas as pd 
import plotly.graph_objects as go
from config_jupyter.config import set_wd
set_wd()

'c:\\Users\\marti\\OneDrive\\Documentos\\Trabajos Indec\\Estimacion-tc-competitivo-Argentina'

In [9]:
cotizaciones = pd.read_excel('./data/cotizaciones 1997.xlsx', sheet_name=0, index_col=0)
itcrm = pd.read_excel('./data/ITCRMSerie.xlsx', header=1, skipfooter=4, index_col="Período", sheet_name=0)

In [10]:
df = cotizaciones[["Estados Unidos"]].merge(itcrm[["ITCRM "]], how="left", right_index=True, left_index=True)
df["dolar_ajustado_itcrm"] = df["Estados Unidos"].iloc[-1] *  df["ITCRM "] / df["ITCRM "].iloc[-1]
df

,Estados Unidos,ITCRM,dolar_ajustado_itcrm
Período,,,
1997-11-24,1.0,83.610626,682.459915
1997-11-25,1.0,83.487422,681.454275
1997-11-26,1.0,83.276353,679.731457
1997-11-27,1.0,83.229786,679.351357
1997-11-28,1.0,83.247999,679.500017
...,...,...,...
2024-03-18,852.5,105.912151,864.492956
2024-03-19,853.0,105.579000,861.773658
2024-03-20,853.0,105.437627,860.619722


In [11]:
inicio = "2003"
fig = go.Figure()
temp = df[inicio:].copy()
fig.add_trace(go.Scatter(x = temp.index, y = temp["dolar_ajustado_itcrm"]))
fig.update_layout(template = None, font_family = "georgia",
                  title_text = "Dólar mayorista ajustado por ITCRM a precios de hoy")
fig.add_annotation(text="Fuente: BCRA. Actualización diaria de @SalvadorVitell1", xref="paper",yref="paper",x=0,y=-0.14,showarrow=False)

In [12]:
from plotly.subplots import make_subplots

In [13]:
def plot_brecha(df:pd.DataFrame, anio_desde='2003', mes_desde='01'):
    # df = df.copy()
    df = df[f'{str(anio_desde)}-{str(mes_desde)}':]
    x = df.index
    ultima_fecha = df.iloc[-1].name
    ultimo_tc_real = round(df["ITCRM "].iloc[-1], 2)

    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(
        name="Tipo de cambio de equilibro", x=x, y=df["ITCRM "], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True, nticks = 15)
    cotizaciones_plot.update_yaxes(
        title_text="ITCRM 17-12-15 = 100", zeroline=False)
    cotizaciones_plot.update_layout(separators=",.", font_family="georgia",
                                    margin ={'b': 50,'l':50,'r':15},
                                    height=600, width=900,
                                    template='none',
                                    title_text=f"Los vaivenes de la política cambiaria<br><sup>Índice de tipo de cambio real {str(anio_desde)}-{str(ultima_fecha.year)}",
                                    title_font=dict(size=20),
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.15, orientation='h'))

    # Flechas
    cotizaciones_plot.add_annotation(x=ultima_fecha, y=ultimo_tc_real, align="left",
                                     text=str(ultimo_tc_real).replace('.', ','), showarrow=True, arrowhead=1)
    
    #Regímnes cambiarios
    anio_desde=int(anio_desde)
    if anio_desde<2007: cotizaciones_plot.add_vline(x="2006-12-12", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_desde<2012: cotizaciones_plot.add_vline(x="2011-10-28", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_desde<2016:cotizaciones_plot.add_vline(x="2015-12-17", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_desde<2019:cotizaciones_plot.add_vline(x="2018-09-26", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_desde<2020:cotizaciones_plot.add_vline(x="2019-09-01", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_desde<2024:cotizaciones_plot.add_vline(x="2023-12-14", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)

    if anio_desde<2004:cotizaciones_plot.add_annotation(showarrow=False, text=f'TC "competitivo y estable"',
        font=dict(size=13), font_family="georgia",x='2004-12-1', y=180)
    if anio_desde<2009:cotizaciones_plot.add_annotation(showarrow=False, text=f'Apreciación cambiaria<br>Inicio intervención INDEC',
        font=dict(size=13), font_family="georgia",x='2009-7-1', y=180)
    if anio_desde<2013:cotizaciones_plot.add_annotation(showarrow=False, text=f'CEPO cambiario<br>INDEC intervenido',
        font=dict(size=13), font_family="georgia",x='2013-11-1', y=180)
    if anio_desde<2017:cotizaciones_plot.add_annotation(showarrow=False, text=f'TC flotante<br>"Flotación sucia"',
        font=dict(size=13), font_family="georgia",x='2017-5-1', y=180)
    if anio_desde<2019:cotizaciones_plot.add_annotation(showarrow=False, text=f"Bandas cambiarias",
        font=dict(size=13), font_family="georgia",x='2019-03-1', y=180,textangle=90)
    if anio_desde<2021:cotizaciones_plot.add_annotation(showarrow=False, text=f'Apreciación cambiaria<br>Con CEPO',
        font=dict(size=13), font_family="georgia",x='2021-9-1', y=180) 
    if anio_desde<2024:cotizaciones_plot.add_annotation(showarrow=False, text=f'Crawling peg 2%<br>Con CEPO',
        font=dict(size=13), font_family="georgia",x='2025-3-1', y=180) 
    
    # Nota al pie
    note = f'Fuente: BCRA. Último dato al {str(ultima_fecha.day)}-{str(ultima_fecha.month)}-{str(ultima_fecha.year)}'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.25, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    cotizaciones_plot.add_hline(y = ultimo_tc_real,line_dash= "dash" )
    return cotizaciones_plot

In [14]:
plot_brecha(df, anio_desde="2002")